In [ ]:
from numpynn import layers, networks, preprocessing, optimizers, losses, metrics, activations
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

### Prepare data
You will need to download the dataset from https://www.kaggle.com/competitions/digit-recognizer/data and place it into the *data* directory.

In [ ]:
data = pd.read_csv('data/mnist/train.csv')
data.head()

In [ ]:
data_enc = preprocessing.categorical_to_numeric(data, columns=['label'])
data_enc.head()

In [ ]:
array = data_enc.to_numpy()
train, val, test = preprocessing.split_train_test_val_data(array, ratio_val=0.1, ratio_test=0.1)

In [ ]:
x_train, y_train = preprocessing.split_X_Y(train, 784)
x_val, y_val = preprocessing.split_X_Y(val, 784)
x_test, y_test = preprocessing.split_X_Y(test, 784)

In [ ]:
x_train = x_train.reshape(x_train.shape[0],1 , 28, -1)
x_val = x_val.reshape(x_val.shape[0], 1, 28, -1)
x_test = x_test.reshape(x_test.shape[0], 1, 28, -1)

print (f'{x_train.shape=}')
print (f'{y_train.shape=}')

print (f'{x_val.shape=}')
print (f'{y_val.shape=}')

print (f'{x_test.shape=}')
print (f'{y_test.shape=}')

Normalization

In [ ]:
x_train = x_train / 255
x_val = x_val / 255
x_test = x_test / 255

### Build the neural network structre

In [ ]:
model = networks.Sequential(input_shape=(1, 28, 28), layers=[
    layers.Convolution(nr_kernels=8, kernel_size=(3, 3), activation=activations.Relu()),
    layers.MaxPooling(pooling_window=(2, 2)),
    layers.Convolution(nr_kernels=16, kernel_size=(3, 3), activation=activations.Relu()),
    layers.MaxPooling(pooling_window=(2, 2)),
    layers.Flatten(),
    layers.Linear(nr_neurons=64, activation=activations.Relu()),
    layers.Linear(nr_neurons=10, activation=activations.Softmax())
])

The network is compiled to internally connect it's layers and initialize the model.

In [ ]:
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-2),
    loss_function=losses.Crossentropy(),
    metric=metrics.Accuracy
)

In [ ]:
model.summary()

### Train the model

In [ ]:
# hist = model.train(x_train, y_train, epochs=10, batch_size=512, verbose=True, val_data=(x_val, y_val))
hist = model.train(x_train, y_train, epochs=50, batch_size=512, verbose=True)

In [ ]:
model.plot_training_loss(hist)

### Evaluate the model

In [ ]:
model.evaluate(x_test, y_test)

### Explore the inner workings
Pick a random image from the testing dataset.

In [ ]:
i = random.randint(0, len(x_test))
image = np.moveaxis(x_test[i], 0, -1)
image_tensor = np.expand_dims(x_test[i], 0)
plt.imshow(image, cmap='gray')

Use it to predict a number and show the probability distribution of the outcome.

In [ ]:
p, _ = model(image_tensor)
plt.bar(np.arange(0, 10), p.reshape(10,))
plt.xlabel('label')
plt.ylabel('probability')

Every layer of the model can be accessed to explore their output. Here we iterate over all the kernels of the convolutional layer to explore what they learned to focus on in images.

In [ ]:
model.plot_conv_activations()